## 套件匯入 

In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np

## 啟動瀏覽器（https://law.judicial.gov.tw/FJUD/default.aspx )

In [10]:
# 司法院官網
link = 'https://law.judicial.gov.tw/FJUD/default.aspx'
options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options)
driver.get(link)

# 定位輸入框，寫入搜尋 <關鍵字>
TEXT = '證券詐欺'
sleep(2)
driver.find_element_by_id('txtKW').send_keys(TEXT)
driver.find_element_by_id('btnSimpleQry').click()

# 等待載入，選擇最高法院（若有）
try:
    driver.find_element_by_xpath("//div[@class='panel-body']   \
    //li/a[contains(text(),'最高法院')]").click()
except NoSuchElementException:
    pass

## 網頁檢索/爬蟲 

In [11]:
# 由於定位資料於 iframe 內
# 因此需要告知瀏覽器切換到 iframe 嵌入框才能進行定位
iframe = driver.find_element_by_xpath("//iframe")
driver.switch_to.frame(iframe)

# 建立須被彈性設定之變數
html_text = list()
status = True

while(status):
    html_text.append(driver.page_source)
    next_ = driver.find_elements_by_id('hlNext')
    if len(next_) != 0:
        next_[0].click()
    else:
        status = False

## 內容擷取

In [12]:
test = list()
for i in html_text:
    soup = BeautifulSoup(i)
    num = 1
    while num < len(soup.find_all('tr')):
        test.append(soup.find_all('tr')[num].text)
        num += 2
test[:10]

['\n1.\n最高法院 109 年度 台上 字第 1719 號刑事判決（29K）\n109.04.15\n違反證券交易法等罪\n',
 '\n2.\n最高法院 108 年度 台上 字第 4056 號刑事判決（29K）\n109.04.15\n違反證券交易法等罪\n',
 '\n3.\n最高法院 108 年度 台上 字第 16 號刑事判決（37K）\n109.01.16\n違反證券交易法等罪\n',
 '\n4.\n最高法院 109 年度 台抗 字第 46 號刑事裁定（5K）\n109.01.09\n違反證券交易法等罪不服再執行羈押\n',
 '\n5.\n最高法院 107 年度 台上 字第 846 號民事判決（11K）\n108.03.27\n請求損害賠償\n',
 '\n6.\n最高法院 108 年度 台抗 字第 355 號刑事裁定（3K）\n108.03.21\n違反證券交易法等罪聲請具保停止羈押\n',
 '\n7.\n最高法院 107 年度 台上 字第 2937 號刑事判決（9K）\n107.10.11\n違反證券交易法等罪\n',
 '\n8.\n最高法院 106 年度 台上 字第 1247 號民事判決（19K）\n107.03.14\n請求侵權行為損害賠償\n',
 '\n9.\n最高法院 106 年度 台上 字第 278 號刑事判決（12K）\n106.09.28\n違反證券交易法\n',
 '\n10.\n最高法院 106 年度 台上 字第 349 號民事判決（17K）\n106.07.26\n請求損害賠償\n']

## 資料清整

In [13]:
df = pd.Series(test).str.split('\n', expand=True)
df1 = df[2].str.split(' ', expand=True)
df2 = df1[6].str.split('（',expand=True).drop(1, axis=1)
df2 = pd.DataFrame(df2[0].str[1:])
df = df.drop([0,1,2,5], axis=1)
df1 = df1.drop([0,2,4,6], axis=1)

In [14]:
df_final = pd.concat([df1, df2, df], axis=1)
df_final.columns = ["年度", "字號", "案號", "類型", "裁判日期", "裁判案由"]
df_final["金額"] = np.random.randint(1000000, 999999999, size=43)

In [15]:
df_final.head()

,年度,字號,案號,類型,裁判日期,裁判案由,金額
0,109,台上,1719,刑事判決,109.04.15,違反證券交易法等罪,61785382
1,108,台上,4056,刑事判決,109.04.15,違反證券交易法等罪,573105930
2,108,台上,16,刑事判決,109.01.16,違反證券交易法等罪,372933046
3,109,台抗,46,刑事裁定,109.01.09,違反證券交易法等罪不服再執行羈押,898763655
4,107,台上,846,民事判決,108.03.27,請求損害賠償,75615095
